<a href="https://colab.research.google.com/github/Lucas-Siade/Data-Science/blob/main/Projeto%20Final/Modelo%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

## **Funções**

### **Carregar os Dados**

In [2]:
def carregarDados(path):
    dataset = pd.read_csv(path, sep="\t")

    dataset = dataset.drop(columns=["<SPREAD>"])
    dataset = dataset.rename(columns={
        "<DATE>":"DATE",
        "<TIME>":"TIME",
        "<OPEN>":"OPEN",
        "<CLOSE>":"CLOSE",
        "<HIGH>":"HIGH",
        "<LOW>":"LOW",
        "<TICKVOL>":"TICKVOL",
        "<VOL>":"VOL"
    })

    dataset["TIMESTAMP"] = pd.to_datetime(dataset["DATE"] + " " + dataset["TIME"], format="%Y.%m.%d %H:%M:%S")
    dataset.drop(columns=["DATE", "TIME"], inplace=True)
    dataset = dataset[["TIMESTAMP", "OPEN", "HIGH", "LOW", "CLOSE", "TICKVOL", "VOL"]]

    return dataset

### **Adicionar dias anteriores**

In [3]:
def adicionarAnteriores(dataset, dias):
    datasetCopy = dataset.copy()

    for i in range(0, dias):
        for coluna in ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL']:
            datasetCopy[f'{coluna}_{i}'] = datasetCopy[coluna].shift(i)

    return datasetCopy

### **Remover valores do dia atual**

In [4]:
def removerValores(dataset):
    colunas = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL']

    return dataset.drop(columns=[col for col in colunas])

## **Modelos**

### **Tendência**

In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/Dados.csv"
dataset = carregarDados(path)
dias = 2

dataset = dataset.copy()
dataset = adicionarAnteriores(dataset, dias)
dataset = dataset.iloc[dias:].reset_index(drop=True)
dataset = removerValores(dataset)

modelTend = joblib.load('/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/modelTend.joblib')
colunasX = [col for col in dataset.columns if any(p in col for p in ['OPEN_', 'HIGH_', 'LOW_', 'CLOSE_', 'TICKVOL_', 'VOL_'])]

probabilidades = modelTend.predict_proba(dataset[colunasX])
dataset["PROBABILIDADE"] = probabilidades[:, 1]*100
dataset["PROBABILIDADE"] = dataset["PROBABILIDADE"].round(2)

In [6]:
display(dataset.head())

,TIMESTAMP,OPEN_0,HIGH_0,LOW_0,CLOSE_0,TICKVOL_0,VOL_0,OPEN_1,HIGH_1,LOW_1,CLOSE_1,TICKVOL_1,VOL_1,PROBABILIDADE
0,2020-10-15 11:51:00,98595,98600,98535,98580,4572,17663,98570.0,98625.0,98570.0,98600.0,5483.0,19111.0,0.00
1,2020-10-15 11:52:00,98575,98620,98535,98615,3536,13143,98595.0,98600.0,98535.0,98580.0,4572.0,17663.0,100.00
2,2020-10-15 11:53:00,98620,98625,98575,98610,3729,13702,98575.0,98620.0,98535.0,98615.0,3536.0,13143.0,0.49
3,2020-10-15 11:54:00,98610,98655,98590,98655,5040,17128,98620.0,98625.0,98575.0,98610.0,3729.0,13702.0,100.00
4,2020-10-15 11:55:00,98650,98665,98610,98615,4021,13669,98610.0,98655.0,98590.0,98655.0,5040.0,17128.0,0.00


### **Prever Valor**

In [7]:
modelPred = joblib.load('/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/modelPred.joblib')
scalerX = joblib.load("/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/scalerX.joblib")
scalerY = joblib.load("/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/scalerY.joblib")

X = dataset[colunasX].values
X = np.concatenate([X, dataset[['PROBABILIDADE']].values], axis=1)
X_scaled = scalerX.transform(X)

Y_pred_scaled = modelPred.predict(X_scaled)
Y_pred = scalerY.inverse_transform(Y_pred_scaled)

dataset['CLOSE_PREDITO'] = Y_pred.flatten()

10274/10274 ━━━━━━━━━━━━━━━━━━━━ 7s 659us/step


## **Tabela Original**

In [21]:
datasetOriginal = carregarDados(path)
datasetOriginal = datasetOriginal.iloc[2:].reset_index(drop=True)

# **Treino do Modelo**

In [32]:
#PARÂMETROS E VARIÁVEIS
gamma = 0.95
epsilon = 1.0
min_epsilon = 0.01
epsilon_decay = 0.995
lr = 0.001
episodes = 25
batch_size = 64
replay_memory_size = 10000
update_target_freq = 10

colunasNumericas = dataset.drop(columns=['TIMESTAMP']).columns
maxValor = dataset[colunasNumericas].max().max()

def get_state(row):
    return torch.tensor([row[col] / maxValor for col in colunasNumericas], dtype=torch.float32)

#MODELO DQN
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(len(colunasNumericas), 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.fc(x)

#INSTÂNCIAS DO MODELO
device = torch.device("cpu")
model = DQN().to(device)
target_model = DQN().to(device)
target_model.load_state_dict(model.state_dict())
target_model.eval()

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

#REPLAY MEMORY
replay_memory = deque(maxlen=replay_memory_size)

#DIVISÃO DOS DADOS
tamanhoTreino = len(dataset) // 5
datasetTreino = dataset.iloc[:tamanhoTreino]
datasetOriginal = datasetOriginal.iloc[:tamanhoTreino]

melhorLucro = float('-inf')
melhorHistorico = []

#TREINO DO MODELO
for ep in range(episodes):
    comprado = False
    priceCompra = 0
    historico = []

    for i in range(len(datasetTreino) - 1):
        row = datasetTreino.iloc[i]
        next_row = datasetTreino.iloc[i + 1]

        state = get_state(row).to(device)
        next_state = get_state(next_row).to(device)

        acoesPossiveis = ['NADA']
        if not comprado:
            acoesPossiveis.insert(0, 'COMPRAR')
        else:
            acoesPossiveis.insert(0, 'VENDER')

        if random.random() < epsilon:
            action = random.choice([0, 1])
        else:
            with torch.no_grad():
                q_values = model(state)
                action = torch.argmax(q_values).item()

        acaoTomada = acoesPossiveis[action]
        recompensa = 0
        info = {}

        if acaoTomada == 'COMPRAR':
            comprado = True
            precoCompra = datasetOriginal.iloc[i]['OPEN']
            info = {'ação': 'COMPRAR', 'preço_compra': precoCompra}

        elif acaoTomada == 'VENDER' and comprado:
            close_predito = row['CLOSE_PREDITO']
            high_real = datasetOriginal.iloc[i]['HIGH']

            if close_predito < high_real:
                precoVenda = close_predito
                recompensa = precoVenda - precoCompra
                info = {
                    'ação': 'VENDER',
                    'preço_compra': precoCompra,
                    'preço_venda': precoVenda,
                    'lucro': recompensa
                }
                comprado = False
                precoCompra = 0
            else:
                acaoTomada = 'NADA'
                info = {'ação': 'NADA'}
        else:
            info = {'ação': 'NADA'}

        historico.append((row['TIMESTAMP'], info))

        replay_memory.append((state, action, recompensa, next_state))

        if len(replay_memory) >= batch_size:
            batch = random.sample(replay_memory, batch_size)
            states, actions, recompensas, next_states = zip(*batch)

            states = torch.stack(states).to(device)
            next_states = torch.stack(next_states).to(device)
            actions = torch.tensor(actions, dtype=torch.long).to(device)
            recompensas = torch.tensor(recompensas, dtype=torch.float32).to(device)

            q_values = model(states).gather(1, actions.unsqueeze(1)).squeeze()

            with torch.no_grad():
                max_target_q_values = target_model(next_states).max(dim=1)[0]

            targets = recompensas + gamma * max_target_q_values

            loss = criterion(q_values, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    if ep % update_target_freq == 0:
        target_model.load_state_dict(model.state_dict())

    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    lucroTotal = sum(info['lucro'] for _, info in historico if info['ação'] == 'VENDER')
    if lucroTotal > melhorLucro:
        melhorLucro = lucroTotal
        melhorHistorico = historico
        melhorModel = model.state_dict()

    print(f"Episódio {ep+1} | Lucro: {lucroTotal:.2f} | ε: {epsilon:.3f}")

torch.save(melhorModel, '/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/modelFinal.pth')

Episódio 1 | Lucro: 191386.89 | ε: 0.995
Episódio 2 | Lucro: 203486.96 | ε: 0.990
Episódio 3 | Lucro: 216314.95 | ε: 0.985
Episódio 4 | Lucro: 220134.19 | ε: 0.980
Episódio 5 | Lucro: 209618.56 | ε: 0.975
Episódio 6 | Lucro: 219519.66 | ε: 0.970
Episódio 7 | Lucro: 223642.14 | ε: 0.966
Episódio 8 | Lucro: 218676.02 | ε: 0.961
Episódio 9 | Lucro: 236655.86 | ε: 0.956
Episódio 10 | Lucro: 233212.36 | ε: 0.951
Episódio 11 | Lucro: 232398.19 | ε: 0.946
Episódio 12 | Lucro: 231415.37 | ε: 0.942
Episódio 13 | Lucro: 240188.00 | ε: 0.937
Episódio 14 | Lucro: 226726.25 | ε: 0.932
Episódio 15 | Lucro: 264117.48 | ε: 0.928
Episódio 16 | Lucro: 254456.78 | ε: 0.923
Episódio 17 | Lucro: 232045.04 | ε: 0.918
Episódio 18 | Lucro: 238764.26 | ε: 0.914
Episódio 19 | Lucro: 255516.91 | ε: 0.909
Episódio 20 | Lucro: 268063.16 | ε: 0.905
Episódio 21 | Lucro: 261935.97 | ε: 0.900
Episódio 22 | Lucro: 258638.43 | ε: 0.896
Episódio 23 | Lucro: 258820.05 | ε: 0.891
Episódio 24 | Lucro: 261994.31 | ε: 0.887
E

# **Aplicação do Modelo**

In [16]:
colunasNumericas = dataset.drop(columns=['TIMESTAMP']).columns
maxValor = dataset[colunasNumericas].max().max()

def get_state(row):
    return torch.tensor([row[col] / maxValor for col in colunasNumericas], dtype=torch.float32)

#MODELO DQN
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(len(colunasNumericas), 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.fc(x)

#INSTÂNCIAS DO MODELO
device = torch.device("cpu")

In [17]:
model = DQN().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/modelFinal.pth', map_location=device))
model.eval()

comprado = False
precoCompra = 0
historicoCompleto = []

for i in range(len(dataset) - 1):
    row = dataset.iloc[i]
    next_row = dataset.iloc[i + 1]

    state = get_state(row).to(device)

    acoesPossiveis = ['NADA']
    if not comprado:
        acoesPossiveis.insert(0, 'COMPRAR')
    else:
        acoesPossiveis.insert(0, 'VENDER')

    with torch.no_grad():
        q_values = model(state)
        action = torch.argmax(q_values).item()

    acaoTomada = acoesPossiveis[action]
    info = {}

    if acaoTomada == 'COMPRAR':
        comprado = True
        precoCompra = datasetOriginal.iloc[i]['OPEN']
        info = {'ação': 'COMPRAR', 'preço_compra': precoCompra}

    elif acaoTomada == 'VENDER' and comprado:
        close_predito = row['CLOSE_PREDITO']
        high_real = datasetOriginal.iloc[i]['HIGH']

        if close_predito < high_real:
            precoVenda = close_predito
            lucro = precoVenda - precoCompra
            info = {
                'ação': 'VENDER',
                'preço_compra': precoCompra,
                'preço_venda': precoVenda,
                'lucro': lucro
            }
            comprado = False
            precoCompra = 0
        else:
            acaoTomada = 'NADA'
            info = {'ação': 'NADA'}
    else:
        info = {'ação': 'NADA'}

    historicoCompleto.append((row['TIMESTAMP'], info))

lucroTotalCompleto = sum(info['lucro'] for _, info in historicoCompleto if info['ação'] == 'VENDER')
print(f"Lucro total no dataset Completo: {lucroTotalCompleto:.2f}")

Lucro total no dataset Completo: 3424969.06


In [18]:
print("\n--- AÇÕES DO MODELO ---")

for tempo, info in historicoCompleto:
    if info['ação'] == 'COMPRAR':
        print(f"[{tempo}] Ação: COMPRAR | Preço: {info['preço_compra']}")
    elif info['ação'] == 'VENDER':
        print(f"[{tempo}] Ação: VENDER | Compra: {info['preço_compra']} → Venda: {info['preço_venda']} | Lucro: {info['lucro']:.2f}")
    else:
        print(f"[{tempo}] Ação: NADA")

print(f"\nLucro total no dataset Completo: {lucroTotalCompleto:.2f}")

A saída de streaming foi truncada nas últimas 5000 linhas.
[2023-04-05 14:51:00] Ação: NADA
[2023-04-05 14:52:00] Ação: NADA
[2023-04-05 14:53:00] Ação: NADA
[2023-04-05 14:54:00] Ação: NADA
[2023-04-05 14:55:00] Ação: NADA
[2023-04-05 14:56:00] Ação: NADA
[2023-04-05 14:57:00] Ação: NADA
[2023-04-05 14:58:00] Ação: NADA
[2023-04-05 14:59:00] Ação: NADA
[2023-04-05 15:00:00] Ação: NADA
[2023-04-05 15:01:00] Ação: NADA
[2023-04-05 15:02:00] Ação: COMPRAR | Preço: 101000
[2023-04-05 15:03:00] Ação: VENDER | Compra: 101000 → Venda: 101223.34375 | Lucro: 223.34
[2023-04-05 15:04:00] Ação: COMPRAR | Preço: 101230
[2023-04-05 15:05:00] Ação: VENDER | Compra: 101230 → Venda: 101265.1875 | Lucro: 35.19
[2023-04-05 15:06:00] Ação: NADA
[2023-04-05 15:07:00] Ação: NADA
[2023-04-05 15:08:00] Ação: NADA
[2023-04-05 15:09:00] Ação: NADA
[2023-04-05 15:10:00] Ação: NADA
[2023-04-05 15:11:00] Ação: NADA
[2023-04-05 15:12:00] Ação: COMPRAR | Preço: 101170
[2023-04-05 15:13:00] Ação: VENDER | Compra: 1

In [19]:
compra_info = None
compra_tempo = None

for tempo, info in historicoCompleto:
    if info['ação'] == 'COMPRAR':
        compra_info = datasetOriginal.loc[datasetOriginal['TIMESTAMP'] == tempo].iloc[0]
        compra_tempo = tempo
    elif info['ação'] == 'VENDER':
        venda_info = datasetOriginal.loc[datasetOriginal['TIMESTAMP'] == tempo].iloc[0]

        print(f"ORDEM DE COMPRA: {compra_tempo}")
        print(f"ORDEM DE VENDA: {tempo}\n")
        print(f"Compra: {info['preço_compra']:.2f} → Venda: {info['preço_venda']:.2f}")
        print(f"Lucro: {info['lucro']:.2f}\n")

        print(f"INFORMAÇÕES DO {compra_tempo} [COMPRA]")
        print(f"OPEN: {compra_info['OPEN']}")
        print(f"LOW: {compra_info['LOW']}")
        print(f"HIGH: {compra_info['HIGH']}")
        print(f"CLOSE: {compra_info['CLOSE']}\n")

        print(f"INFORMAÇÕES DO {tempo} [VENDA]")
        print(f"OPEN: {venda_info['OPEN']}")
        print(f"LOW: {venda_info['LOW']}")
        print(f"HIGH: {venda_info['HIGH']}")
        print(f"CLOSE: {venda_info['CLOSE']}")
        linha_predito = dataset.loc[dataset['TIMESTAMP'] == tempo].iloc[0]
        print(f"CLOSE_PREDITO: {linha_predito['CLOSE_PREDITO']}\n")
        print(70*'-')

        compra_info = None
        compra_tempo = None

A saída de streaming foi truncada nas últimas 5000 linhas.
ORDEM DE COMPRA: 2023-04-17 16:11:00
ORDEM DE VENDA: 2023-04-17 16:13:00

Compra: 107670.00 → Venda: 107774.76
Lucro: 104.76

INFORMAÇÕES DO 2023-04-17 16:11:00 [COMPRA]
OPEN: 107670
LOW: 107635
HIGH: 107710
CLOSE: 107690

INFORMAÇÕES DO 2023-04-17 16:13:00 [VENDA]
OPEN: 107710
LOW: 107705
HIGH: 107785
CLOSE: 107755
CLOSE_PREDITO: 107774.7578125

----------------------------------------------------------------------
ORDEM DE COMPRA: 2023-04-17 16:14:00
ORDEM DE VENDA: 2023-04-17 16:15:00

Compra: 107760.00 → Venda: 107793.43
Lucro: 33.43

INFORMAÇÕES DO 2023-04-17 16:14:00 [COMPRA]
OPEN: 107760
LOW: 107725
HIGH: 107760
CLOSE: 107750

INFORMAÇÕES DO 2023-04-17 16:15:00 [VENDA]
OPEN: 107750
LOW: 107750
HIGH: 107795
CLOSE: 107795
CLOSE_PREDITO: 107793.4296875

----------------------------------------------------------------------
ORDEM DE COMPRA: 2023-04-17 16:17:00
ORDEM DE VENDA: 2023-04-17 16:18:00

Compra: 107775.00 → Venda: 1

In [20]:
dadosTabela = []
tempo_compra = None

for tempo, info in historicoCompleto:
    ordem = info.get('ação', 'NADA').upper()
    lucro = info['lucro'] if ordem == 'VENDER' else None
    duracao = None

    if ordem == 'COMPRAR':
        tempo_compra = tempo
    elif ordem == 'VENDER' and tempo_compra is not None:
        try:
            duracao = pd.to_datetime(tempo) - pd.to_datetime(tempo_compra)
            duracao = duracao.total_seconds() / 60
        except:
            duracao = None
        tempo_compra = None

    dadosTabela.append({
        'TIMESTAMP': tempo,
        'ORDEM': ordem,
        'LUCRO': lucro,
        'DURACAO (MINUTOS)': duracao
    })

tabela = pd.DataFrame(dadosTabela)
display(tabela.tail())

tabela.to_csv(
    "/content/drive/MyDrive/Colab Notebooks/Data Science/Projeto Final/Resultado.csv",
    index=False,
    sep=';',
    decimal=','
)

,TIMESTAMP,ORDEM,LUCRO,DURACAO (MINUTOS)
328754,2023-04-19 17:49:00,NADA,NaN,NaN
328755,2023-04-19 17:50:00,NADA,NaN,NaN
328756,2023-04-19 17:51:00,VENDER,107.203125,4.0
328757,2023-04-19 17:52:00,NADA,NaN,NaN
328758,2023-04-19 17:53:00,NADA,NaN,NaN
